In [ ]:
import os, requests
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import datetime
import time
import math
import re
from pycoingecko import CoinGeckoAPI
from huobi import HuobiRestClient
from discord_webhook import DiscordWebhook

from datetime import datetime

from uniswap import Uniswap
from web3 import Web3, HTTPProvider

from multiprocessing import Process
from threading import Thread

In [ ]:
#Defining environments for Discord Notification
home = os.environ['HOME']
java_home = os.environ.get('JAVA_HOME')

url_wb = os.environ.get('DISCORD_WH')

In [ ]:
#MetaMask Authentication for Uniswap interaction
address = "0xD....53"          # or "0x0000000000000000000000000000000000000000", if you're not making transactions
private_key = "d...5"  # or None, if you're not going to make transactions
uniswap_wrapper = Uniswap(address, private_key, provider= 'https://mainnet.infura.io/v3/3...b', version=2)  # pass version=2 to use Uniswap v2
eth = "0x0000000000000000000000000000000000000000"
eth_balance = uniswap_wrapper.get_eth_balance()*0.80

In [ ]:
#Private Huobi Account Details (Providing access to the exchange)
huobi_access_key = '0...3'
huobi_secret_key = 'e...0'
client = HuobiRestClient(access_key=huobi_access_key, secret_key=huobi_secret_key)
huobi_account_id=client.accounts().data['data'][0]['id']

In [ ]:
#Bag of exchanges integrated so far! Format: Coingecko
#for reference only, we are not using this variable
my_exchanges = ['Huobi Global', 'Uniswap (v2)']

In [ ]:
#initialize APIs
cg = CoinGeckoAPI()

In [ ]:
#Update your database
url = "https://www.binance.com/en/support/announcement/c-48?navId=48#/48"
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')
news_list = soup.find_all(class_ = 'css-1neg3js')

comprabale_news = []
for news in news_list:
    comprabale_news.append(news.text)
    
print('Your Past Data', comprabale_news)

In [ ]:
#DEFINING HUOBI STOP LOSS FUNCTION
def huobi_stop_loss_fun():
    global huobi_price
    global huobi_sl
    global first_id_candle
    global stop_loss_percentage
    global huobi_spendable_balance_new
    global huobi_pos_end
    while huobi_price > huobi_sl:
        try:
            huobi_price = client.market_history_trade(symbol=symbol, size=1).data['data'][0]['data'][0]['price']
        except:
            print('Could not fetch the last market trade')
            pass
        print('the huobi price is:', huobi_price, 'while your huobi SL is:', huobi_sl , datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        time.sleep(1)
        try:
            if first_id_candle != client.market_history_kline(symbol=symbol, period='5min', size=1).data['data'][0]['id']:
                time.sleep(1)
                last_candle_open=client.market_history_kline(symbol=symbol, period='5min', size=2).data['data'][1]['open']
                last_candle_close=client.market_history_kline(symbol=symbol, period='5min', size=2).data['data'][1]['close']
                mid_price = (last_candle_close + last_candle_open)/2
                first_id_candle = client.market_history_kline(symbol=symbol, period='5min', size=1).data['data'][0]['id']
                new_huobi_sl = mid_price*stop_loss_percentage
                print('A wild 5min candle appeared!!')
                if huobi_sl < new_huobi_sl:
                    huobi_sl = mid_price*stop_loss_percentage
                    print('New Huobi SL defined')
        except:
            print('An Huobi API error occurred')
            pass
    try:
        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance_new, symbol=symbol, type='sell-market').data
        print('Sold')
    except:
        print('Couldnt sell Huobi position')
        pass
    huobi_pos_end = client.market_history_trade(symbol=symbol, size=1).data['data'][0]['data'][0]['price']
    print('the Huobi close prize was:', huobi_pos_end)

In [ ]:
def uniswap_buy_action():
    global eth
    global target_address
    global eth_balance
    uniswap_wrapper.make_trade(Web3.toChecksumAddress(eth), Web3.toChecksumAddress(target_address), qty=int(eth_balance))
    print('Thread 2 reached an end')

In [ ]:
#BINANCE EYE (CHECK FOR NEWS UPDATE EVERY 0.1 SECONDS)
i=0
stop_loss_percentage = 0.94
#Checks for the next 10000000 news and acts uppon them
while i<10000000:
    try:
        while comprabale_news[0] == BeautifulSoup(get(url).text, 'html.parser').find_all(class_ = 'css-1neg3js')[0].text:
            time.sleep(0.1)
            print('No new listings', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
            
        #Parsing the recent new to see if it's interesting
        print('Hey Something is different')
        print('CP1 - Recognized a brand new Announcement')
        recent_new = BeautifulSoup(get(url).text, 'html.parser').find_all(class_ = 'css-1neg3js')[0].text
        comprabale_news[0]=recent_new
        print(recent_new)
        #recent_new = 'Binance Will List Celo (CELO)'
        
        if ('List ' in recent_new) or ('Lists ' in recent_new):
            print('CP2 - Recognized the word List or Lists inside the recent_new')
            print('filtering the name & ticker...')
            ticker=re.findall('\((\d*\w*\d*\w*)\)', recent_new)[0].lower()
            name = re.findall('Lists? [\w\d]* ?[\w\d]* ?[\w\d]*', recent_new)[0].split()[1:]
            first_name = name[0]
            if len(name) == 1:
                name = name[0].lower()
            elif len(name) > 1:
                name = '-'.join(name).lower()
            print(name, ticker, 'is our listing today')
            print('Gathering Markets via Coingecko API...')
            df=pd.DataFrame(cg.get_coins_list())
            df = df[df['symbol']==ticker.lower()].reset_index(drop=True)
            if len(df)==0:
                raise SystemExit("Could not find Coin on COINGECKO")
            elif len(df) > 1:
                for value in df['name']:
                    if first_name in value:
                        unique_id = df[df['name']==value]['id'].reset_index(drop=True)[0]
            elif len(df)==1:
                unique_id = df['id'][0]
            print('CP3')
            exchanges_available = []
            for data in cg.get_coin_by_id(id=unique_id)['tickers']:
                exchanges_available.append((data['market']['name'], data['target']))
            print('CP4')
            df_pairs = pd.DataFrame(exchanges_available, columns=['Exchange','Pair'])
            print('CP5')
            ###EXCHANGE MATCHING
            try:
                target_address = cg.get_coin_by_id(id=unique_id)['contract_address']
            except:
                print("It's not an ERC-20 token")
            print('CP6 - Gathered all data required for start trading')
            if 'Uniswap (v2)' in list(df_pairs['Exchange']):
                try:
                    #uniswap_buy_action()
                    #t2 = Thread(target = uniswap_buy_action)
                    #t2.setDaemon(True)
                    #t2.start()
                    
                    # This transaction needs to custmize gas fee according to ETH network usage
                    
                    print('Uni Thread ended')
                except:
                    print('Something went wrong trying to buy at Uniswap')
                    pass
            else:
                print("This Coin isn't tradable on Uniswap")
            print('CP7')
            if 'Huobi Global' in list(df_pairs['Exchange']):
                available_pairs = list(df_pairs[df_pairs['Exchange']=='Huobi Global']['Pair'])
                if 'BTC' in available_pairs:
                    try:
                        symbol = ticker+'btc'
                        print('We already have BTC')
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance=str(round(float(df[(df['currency']=='btc')&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #Market Taker Buy order
                        print('Time of huobi buy :', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
                        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance, symbol=symbol, type='buy-market').data
                        huobi_price=client.market_history_trade(symbol=symbol, size=1).data['data'][0]['data'][0]['price']
                        first_id_candle = client.market_history_kline(symbol=symbol, period='5min', size=1).data['data'][0]['id']
                        #Check Balance for the new crypto that was just bought
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance_new=str(round(float(df[(df['currency']==ticker)&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #STOP LOSS IMPLEMENTED
                        huobi_sl = huobi_price*stop_loss_percentage
                        print('the open price was', huobi_price)
                        #huobi_stop_loss_fun()
                        t1 = Thread(target = huobi_stop_loss_fun)
                        t1.setDaemon(True)
                        t1.start()
                    except:
                        print('I cant trade for some reason at Huobi')
                        pass
                elif 'ETH' in available_pairs:
                    try:
                        symbol = ticker+'eth'
                        print('Converting BTC to ETH')
                        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance, symbol='ethbtc', type='buy-market').data
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance=str(round(float(df[(df['currency']=='eth')&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #Market Taker Buy order
                        print('Time of huobi buy :', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
                        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance, symbol=symbol, type='buy-market').data
                        huobi_price=client.market_history_trade(symbol=symbol, size=1).data['data'][0]['data'][0]['price']
                        first_id_candle = client.market_history_kline(symbol=symbol, period='5min', size=1).data['data'][0]['id']
                        #Check Balance for the new crypto that was just bought
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance_new=str(round(float(df[(df['currency']==ticker)&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #STOP LOSS IMPLEMENTED
                        huobi_sl = huobi_price*stop_loss_percentage
                        print('the open price was', huobi_price)
                        #huobi_stop_loss_fun()
                        t1 = Thread(target = huobi_stop_loss_fun)
                        t1.setDaemon(True)
                        t1.start()
                    except:
                        print('I cant trade for some reason at Huobi')
                        pass
                elif 'USDT' in available_pairs:
                    try:
                        symbol = ticker+'usdt'
                        print('Converting BTC to USDT')
                        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance, symbol='btcusdt', type='sell-market').data
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance=str(round(float(df[(df['currency']=='usdt')&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #Market Taker Buy order
                        print('Time of huobi buy :', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
                        client.place(account_id=huobi_account_id, amount=huobi_spendable_balance, symbol=symbol, type='buy-market').data
                        huobi_price=client.market_history_trade(symbol=symbol, size=1).data['data'][0]['data'][0]['price']
                        first_id_candle = client.market_history_kline(symbol=symbol, period='5min', size=1).data['data'][0]['id']
                        #Check Balance for the new crypto that was just bought
                        balance=client.balance(account_id=huobi_account_id).data['data']['list']
                        df=pd.DataFrame.from_dict(balance)
                        huobi_spendable_balance_new=str(round(float(df[(df['currency']==ticker)&(df['type']=='trade')]['balance'].reset_index(drop=True)[0])*0.99, 5))[:-1]
                        #STOP LOSS IMPLEMENTED
                        huobi_sl = huobi_price*stop_loss_percentage
                        print('the open price was', huobi_price)
                        #huobi_stop_loss_fun()
                        t1 = Thread(target = huobi_stop_loss_fun)
                        t1.setDaemon(True)
                        t1.start()
                    except:
                        print('I cant trade for some reason at Huobi')
                        pass
            else:
                print("This Coin isn't tradable on Huobi")
            i+=1
            print('CP8')
            #if 'KuCoin' in list(df_pairs['Exchange']):
            #####
            
    ###### WRITE TO CSV
        try:
            print('CP9')
            if ('List ' in recent_new) or ('Lists ' in recent_new):
                article_link = 'https://www.binance.com' + BeautifulSoup(get(url).text, 'html.parser').find_all(class_ = 'css-1neg3js')[0].get('href')
                webhook = DiscordWebhook(url=url_wb, content=recent_new +'\n'+ article_link)
                response = webhook.execute()
                print('Notified on Discord')
                
            # Open old database file
                print('CP10')
                path = "C...........xlsx"
                df_database = pd.read_excel(path)
                order_filled = 0
                uni_status = 0
                huobi_status = 0
                price = cg.get_price(ids=unique_id, vs_currencies='usd')[unique_id]['usd']
                new_data = [datetime.now().strftime("%d/%m/%Y"), datetime.now().strftime("%H:%M:%S"), recent_new, article_link, ticker, name, price, ", ".join(set(list(df_pairs['Exchange']))), order_filled, uni_status, huobi_status, "Type Description", "Type ROI" ]
                cols = ['Day','Time', 'Title', 'URL', 'Ticker', 'Name', 'Price', 'Exchanges', 'Order Fulfillment', 'Uni_Status', 'Huobi_Status', 'Description', 'ROI']
                df_database = df_database.append(pd.Series(new_data, index=df_database.columns), ignore_index=True)
                df_database.to_excel(path, index = False)
                print('New listing send to CSV')
            else:
                print('Not an Interesting new')
        except:
            print('Couldnt not write on DB')
        #if ('List ' in recent_new) or ('Lists ' in recent_new):
        #    raise SystemExit("Our trading is done")
    except:
        print("Couldn't fetch Binance News Or Coingecko tickers! PLEASE HELP")
        i+=1
        pass
        
    print('Reached the END of the while loop:', i)
    
print('Reached the END of script')



In [ ]:
help(HuobiRestClient)